In [ ]:
    mypartialthread = run.RosettaRun(rosetta_bin='/home/bertolan/rosetta_new/Rosetta/main/source/bin',rosetta_db='/home/bertolan/rosetta_new/Rosetta/main/database',binary='partial_thread.default.linuxgccrelease')
    origseq = mymaindir+'/sequences/'+name+'.fasta'
    mypartialthread.add_flags('-in:file:fasta %s' %origseq)
    mypartialthread.add_flags('-in:file:alignment alignment.grishin')

    try:
        os.remove('alignment.grishin')
    except:
        pass
    mytemp = list()
    for each in mytemplates:
        each.download_pdb()
        each.download_seq()
        each.show_my_aln(make_file=True) #writes each template grishin alignmnet to file
        mytemp.append(each.pdbfile[:-3])

    s = ' '
    spdb = s.join(mytemp)
    mypartialthread.add_flags('-in:file:template_pdb %s' %spdb )
    mypartialthread.add_flags('-ignore_unrecognized_res T')
    mypartialthread.add_flags('-overwrite T')
    print " Running Partial threading of target sequence onto templates found"
    mypartialthread.run()

    os.chdir(mymaindir)
    os.chdir(mymodelspath)
    if not os.path.isdir(name):
        os.mkdir(name)
    os.chdir(name)

    if not os.path.isdir('model'):
        os.mkdir('model')
    os.chdir('model')

       ## now calc the weight for each model, setting the largest equal to 1

    print "Writing qsub engine submission for the hybridize run"
    with open('epiph_%s.sh' %name, 'wb') as fh:
        myhybridizerun = run.RosettaRun(rosetta_bin='/home/bertolan/rosetta_new/Rosetta/main/source/bin',rosetta_db='/home/bertolan/rosetta_new/Rosetta/main/database',binary='rosetta_scripts.default.linuxgccrelease')
        myhybridizerun.add_flags('@flags')
        myhybridizerun.add_flags('-in:file:fasta %s' %origseq)
        myhybridizerun.add_flags('-parser:protocol hybridize.xml')
        myhybridizerun.add_flags('-out:file:silent %s.out' %name)
        myhybridizerun.add_flags('-frag3 %s' %name[0:4]+'_.200.3mers' )
        myhybridizerun.add_flags('-frag9 %s' %name[0:4]+'_.200.9mers')

        fh.write( myhybridizerun.test_run() )


    write_hybridize_flags()
    write_stage1_wts('stage1.wts')
    write_stage2_wts('stage2.wts')
    write_stage3_wts('stage3.wts')
    write_xml(mytemplates,mytemplatedir)



    print "About to run James Thompson Constraints on templates"
    mycmd = "%s/cm_scripts/bin/predict_distances_multi.pl %s/alignment.grishin %s/%s  -outfile alignment.grishin.dist_csts -min_seqsep 5 -max_dist 10 -aln_format grishin -max_e_value 10000 -pdb_dir %s " %(environ['ROBETTA'],(mymaindir+'/templates/'+name), (mymaindir+'/sequences'), ( name +'.fasta'),mytemplatedir)
    mytmp = (mymaindir+'/templates/'+ name)
    mycmd2 = 'grep " CA " alignment.grishin.dist_csts.bb_sc > alignment.grishin.dist_csts.bb_sc.CA'
    mycmd3 = 'grep " CA " alignment.grishin.dist_csts > alignment.grishin.dist_csts.CA'
    print mycmd

    if not isfile('%s/alignment.grishin.dist_csts' %mytemp):
        try:
            os.system(mycmd)
            os.system(mycmd2)
            os.system(mycmd3)
        except OSError as e:
            print "error running predict_distances_multi.pl (James Thompson Evolutionary Constraints) \n %s" %e

    myfragname = name[:4]+'_'
    print "About to make Fragments for the target sequence"
    if (nofrags == False):
        if not isfile( (myfragname+'.make_fragments.success') ):
# Once I get the new one to work, use this
            mycmdfrag = '/share/backup/bertolan/frag_picker_do_not_touch/Rosetta/tools/fragment_tools/make_fragments.pl -verbose -id %s %s ' %(myfragname,(mymaindir+'/sequences/'+name+'.fasta'))
#        mycmdfrag = '/share/tmp-data-1/siegellab/frag_picker_do_not_touch/Rosetta/tools/fragment_tools/make_fragments.pl -nohoms -verbose -id %s %s ' %(myfragname,(mymaindir+'/sequences/'+name+'.fasta'))
            print mycmdfrag
#should add a try 5 times before dying here
            try:
                os.system(mycmdfrag)#? why this and then the next one?
#            subprocess.Popen(mycmdfrag,shell=True)
            except OSError as e:
                print "uh oh! error generating fragments; good luck, maybe you should just have a beer \n %s" %e
    else:
        print "Not making fragments per your request, set nofrags=False to change this"
            #each.update_seq_with_pdb_seq() ## Doesn't appear to be different for my 1 test case, can add in later, just update and then realign
        ## realign new sequences that match PDBS, then write out all the files needed/ convert format to grishin, write xml, rosetta flags, etc


if __name__ == '__main__':
    main()
